# Running cisTopic with input features from ScreSeg-fi on multiome ATAC modality

For dimensionality reduction and clustering of scATAC-seq data we found that using Wolfgang's ScregSeg model and cisTopic gave us the best clustering resolution. I want to try this with the multiome-ATAC modality.

I made a few adaptions of the standard 10X scATAC tutorial to get this to work with the ATAC modality of multiome data: http://htmlpreview.github.io/?https://github.com/aertslab/cisTopic/blob/master/vignettes/10X_workflow.html

## Set up installation

In [ ]:
# set project head directory
project_path<-"/fish/"
# Specify sample name
sample<-"multiome-fish"
# extract metadata
metadata<-read.csv(paste(project_path,"sample_metadata.csv", sep=""))
#metadata
sample_metadata<-metadata[metadata$Sample_Name==sample,]

In [ ]:
# already installed devtools::install_github("aertslab/cisTopic") with envrionment
library(cisTopic)
library(GenomicRanges)
lapply(c('Rsubread', 'umap', 'Rtsne', 'ComplexHeatmap', 
         'fastcluster', 'data.table', 'rGREAT', 'ChIPseeker', 
         'TxDb.Hsapiens.UCSC.hg19.knownGene', 'org.Hs.eg.db', 
         'densityClust'),
       require, character.only = TRUE)


## Input data
From cisTopic manual
Starting from the CellRanger ATAC fragments file and a bed file with potential rgeulatory regions. For more information regards the fragments file, please visit: https://support.10xgenomics.com/single-cell-atac/software/pipelines/latest/output/fragments. 

I will actually use the informative states generated by ScregSeg and exported as a bed file 'informative_12_rarest_states.bed'

The scregseg output bed file has several columns in addition to chr, start, end. These need to be removed for compatibility with cistopic so I removed them with the following awk command.

In [ ]:
# the cellranger peak bed files have only chr, start and end so I need to parse this 
#system("awk -F'\t' '{print $1"\t"$2"\t"$3}' informative_12_rarest_states.bed > informative_12_rarest_states.3col.bed")

To create a cisTopic obeject some metrics information that is output from cellranger is used. The way this is used is available on the github page [https://github.com/aertslab/cisTopic/blob/04cecbb9d1112fcc1a6edc28b5a506bcb49f2803/R/InitializecisTopic.R](https://github.com/aertslab/cisTopic/blob/04cecbb9d1112fcc1a6edc28b5a506bcb49f2803/R/InitializecisTopic.R) This serves two purposes:
1. Selection of barcodes called as cells
2. Additional metrics in the cisTopic object

However the metrics file output by cellranger-arc (for multiome) has a slightly different format because it includes information on both modalities. So I selected the colums that match the columns found in the scATAC metrics files, and renamed these to match. NB a few columns were missing, these seem to mainly be addition metrics e.g. blacklist or DNase sensitive regions so I hope they will just be optional downstream.

In [ ]:
# read in metrics file from multiome
metrics_preparse<-read.csv(metrics)

#select columns from that match the metrics from scATAC
metrics_parsed<-metrics_preparse[c('barcode', 'atac_raw_reads','atac_dup_reads','atac_chimeric_reads',
                                   'atac_unmapped_reads','atac_lowmapq','atac_mitochondrial_reads', 
                                   #no "passed_filters" or "cell_id"
                                   'is_cell', 'atac_TSS_fragments', 
                                   # no DNase_sensitive_region_fragments, no enhancer_region_fragments, no promoter_region_fragments
                                   # no on_target_fragments, blacklist_region_fragments
                                   'atac_peak_region_fragments','atac_peak_region_cutsites')
                                 ]
colnames(metrics_parsed)<-c('barcode','total','duplicate','chimeric',
                            'unmapped','lowmapq','mitochondrial',
                            #'passed_filters','cell_id',
                            'is__cell_barcode','TSS_fragments',
                            #'DNase_sensitive_region_fragments','enhancer_region_fragments','promoter_region_fragments',
                            #'on_target_fragments','blacklist_region_fragments',
                            'peak_region_fragments','peak_region_cutsites')
metrics_parsed_path<-paste0(pathTo10X, '/per_barcode_metrics_parsed.csv')
write.csv(metrics_parsed, 
         file=metrics_parsed_path, quote=FALSE, row.names=FALSE)

Now set up the paths for the cisTopic object as in the tutorial but using 'per_barcode_metrics_parsed.csv' instead of 'per_barcode_metrics.csv'

In [ ]:
pathTo10X <- paste0(project_path,sub("./", "", sample_metadata$cellranger_count_out_path))
fragments <- paste0(pathTo10X, '/atac_fragments.tsv.gz')
regions <- 'informative_12_rarest_states.3col.bed'
#metrics <- paste0(pathTo10X, '/per_barcode_metrics.csv')
metrics_parsed_path<-paste0(pathTo10X, '/per_barcode_metrics_parsed.csv')
# check whether this will work because format of metrics might be different for multiome
cisTopicObject <- createcisTopicObjectFrom10X(fragments, regions, metrics_parsed_path, project.name=sample)

This works and the dimensionality reduction can be performed as in the tutorial.